In [ ]:
pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
import evaluate
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification
)
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {0: "negative", 1: "neutral", 2: "positive"}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/test.csv")

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("UWB-AIR/Czert-B-base-cased")

def tokenize_function(example):
    question_string = f"What is the sentiment of aspect: {example['aspect']}?"

    encoding = tokenizer(
        text=example["text"],
        text_pair=question_string,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    # Convert label from string to int
    encoding["label"] = label2id[example["label"]]
    return encoding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/214k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=False)
val_dataset = val_dataset.map(tokenize_function, batched=False)
test_dataset = test_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/2521 [00:00<?, ? examples/s]

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/361 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "UWB-AIR/Czert-B-base-cased",
    num_labels=3
)

pytorch_model.bin:   0%|          | 0.00/441M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UWB-AIR/Czert-B-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=200,
    warmup_ratio=0.06,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fialavi2 (fialavi2-czech-technical-university-in-prague) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.756105,0.675000
2,0.882200,0.681435,0.713889
3,0.596600,0.687552,0.743056
4,0.384300,0.799923,0.755556
5,0.384300,0.857547,0.756944
6,0.221900,1.127517,0.758333
7,0.125100,1.205594,0.772222
8,0.089400,1.358758,0.761111
9,0.042900,1.463664,0.752778
10,0.042900,1.461467,0.762500


TrainOutput(global_step=1580, training_loss=0.29854578805875176, metrics={'train_runtime': 1440.3469, 'train_samples_per_second': 17.503, 'train_steps_per_second': 1.097, 'total_flos': 3316544630461440.0, 'train_loss': 0.29854578805875176, 'epoch': 10.0})

In [ ]:
val_results = trainer.evaluate()
print(val_results)

{'eval_loss': 1.2055937051773071, 'eval_accuracy': 0.7722222222222223, 'eval_runtime': 10.7078, 'eval_samples_per_second': 67.241, 'eval_steps_per_second': 4.203, 'epoch': 10.0}


In [ ]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 1.2137470245361328, 'eval_accuracy': 0.7423822714681441, 'eval_runtime': 5.4789, 'eval_samples_per_second': 65.889, 'eval_steps_per_second': 4.198, 'epoch': 10.0}


In [ ]:
trainer.save_model("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/Czert-B-base-casedQA")

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/Czert-B-base-casedQA")

('/content/drive/MyDrive/Newton/SemEval-Restaurants/models/Czert-B-base-casedQA/tokenizer_config.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/Czert-B-base-casedQA/special_tokens_map.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/Czert-B-base-casedQA/vocab.txt',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/Czert-B-base-casedQA/added_tokens.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/Czert-B-base-casedQA/tokenizer.json')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/Czert-B-base-casedQA")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/Czert-B-base-casedQA")

## Vyhodnocovací metriky

In [ ]:
def compute_metrics_accuracy(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer_accuracy = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_accuracy
)

In [ ]:
test_results_accuracy = trainer_accuracy.evaluate()
print(test_results_accuracy)

{'eval_loss': 1.213747262954712, 'eval_model_preparation_time': 0.0031, 'eval_accuracy': 0.7423822714681441, 'eval_runtime': 5.5241, 'eval_samples_per_second': 65.35, 'eval_steps_per_second': 8.327}


In [ ]:
def compute_metrics_weightedf1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

trainer_weightedf1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_weightedf1
)

In [ ]:
test_results_weightedf1 = trainer_weightedf1.evaluate()
print(test_results_weightedf1)

{'eval_loss': 1.213747262954712, 'eval_model_preparation_time': 0.0028, 'eval_f1': 0.740228544953797, 'eval_runtime': 5.6448, 'eval_samples_per_second': 63.952, 'eval_steps_per_second': 8.149}


In [ ]:
def compute_metrics_macrof1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

trainer_macrof1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_macrof1
)

In [ ]:
test_results_macrof1 = trainer_macrof1.evaluate()
print(test_results_macrof1)

{'eval_loss': 1.213747262954712, 'eval_model_preparation_time': 0.0061, 'eval_f1': 0.673036909768168, 'eval_runtime': 5.2385, 'eval_samples_per_second': 68.913, 'eval_steps_per_second': 8.781}


## Praktické testy

In [ ]:
def predict_sentiment(model, tokenizer, aspect, text):
    question_string = f"What is the sentiment of aspect: {aspect}?"

    encoding = tokenizer(
        text=text,
        text_pair=question_string,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256
    )

    with torch.no_grad():
        outputs = model(**encoding)

    logits = outputs.logits
    print(logits)
    predicted_class_id = logits.argmax(dim=1).item()
    return id2label[predicted_class_id]

### Sada L

In [ ]:
example_aspect = "power"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L1: {prediction}, expected: positive")
example_aspect = "display"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L2: {prediction}, expected: negative")
example_aspect = "battery"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L3: {prediction}, expected: positive")

tensor([[-2.6873, -2.8176,  5.3076]])
Predicted sentiment for L1: positive, expected: positive
tensor([[ 3.3401, -3.3175, -0.2029]])
Predicted sentiment for L2: negative, expected: negative
tensor([[-1.4416, -3.5777,  4.6410]])
Predicted sentiment for L3: positive, expected: positive


### Sada R

In [ ]:
example_aspect = "staff"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R1: {prediction}, expected: negative")
example_aspect = "table"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R2: {prediction}, expected: negative")
example_aspect = "food"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R3: {prediction}, expected: positive")
example_aspect = "beer"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R4: {prediction}, expected: positive")

tensor([[ 1.9524, -4.0080,  1.4706]])
Predicted sentiment for R1: negative, expected: negative
tensor([[ 0.6132, -3.5945,  2.4232]])
Predicted sentiment for R2: positive, expected: negative
tensor([[-2.6386, -2.9217,  5.3920]])
Predicted sentiment for R3: positive, expected: positive
tensor([[-2.6260, -3.0028,  5.4000]])
Predicted sentiment for R4: positive, expected: positive


### Sada M

In [ ]:
example_aspect = "Osaka"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M1: {prediction}, expected: neutral")
example_aspect = "Saremi"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M2: {prediction}, expected: positive")
example_aspect = "Babtel"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M3: {prediction}, expected: negative")

tensor([[-2.5771,  3.7268, -1.1201]])
Predicted sentiment for M1: neutral, expected: neutral
tensor([[ 0.1503, -3.1146,  2.0744]])
Predicted sentiment for M2: positive, expected: positive
tensor([[ 2.5400, -1.1340, -1.8530]])
Predicted sentiment for M3: negative, expected: negative


### Sada O

In [ ]:
example_aspect = "day"
example_text = "Today was a really good day, Anna asked me out for a date! Unfortunately the weather was awful, so we had to cancel it."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O1: {prediction}, expected: positive")
example_aspect = "weather"
example_text = "Today was a really good day, Anna asked me out for a date! Unfortunately the weather was awful, so we had to cancel it."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O2: {prediction}, expected: negative")

tensor([[-0.6241, -1.5475,  2.4177]])
Predicted sentiment for O1: positive, expected: positive
tensor([[ 4.5293, -3.2172, -1.8411]])
Predicted sentiment for O2: negative, expected: negative
